In [33]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from tabulate import tabulate
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
from scipy.stats import chi2
from sklearn.preprocessing import StandardScaler
from stargazer.stargazer import Stargazer
import os

In [34]:
# File Paths
absolute_path = "C:\\Users\\tykun\\\OneDrive\\Documents\\SchoolDocs\VSCodeProjects\\connectedData\\board_analysis\\"
altered_dataframes = "altered_dataframes\\"
gpt_dataframes = "gpt_dataframes\\"
graphs = "graphs\\"
scripts =  "scripts\\"
board_dataframes = "board_dataframes\\"
temporary = "temporary_data\\"
final_scripts = "final_scripts\\"
normalized_dataframes = "normalized_dataframes\\"
college_matching = "college_matching\\"

altered_dataframe_path = f"{absolute_path}{altered_dataframes}"
gpt_dataframe_path = f"{absolute_path}{gpt_dataframes}" 
graph_path = f"{absolute_path}{graphs}"
script_path = f"{absolute_path}{scripts}"
boards_path = f"{absolute_path}{board_dataframes}"
temporary_data_path = f"{absolute_path}{temporary}"
college_matching_path = f"{absolute_path}{college_matching}"


# Valid Years
years = ["1999", "2000", "2005", "2007", "2008", "2009", "2011", "2013", "2018"]

#Created Files
university_boards_statistics_path = f"{altered_dataframe_path}sample_board_statistics.csv"
university_admissions_path = f"{college_matching_path}university_admission_rate.csv"
university_demographics_path = f"{college_matching_path}university_student_demographics.csv"
university_faculty_path = f"{college_matching_path}university_faculty.csv"

In [35]:
university_board_statistics_df = pd.read_csv(university_boards_statistics_path)
updated_stats_path = os.path.join(absolute_path, final_scripts, normalized_dataframes, "normalized_university_board_statistics.csv")
# university_board_statistics_df = pd.read_csv(updated_stats_path)
university_admissions_df = pd.read_csv(university_admissions_path)
university_demographics_df = pd.read_csv(university_demographics_path)
university_faculty_df = pd.read_csv(university_faculty_path)

In [36]:
def remove_non_samples(df):
    df = df[df['PrimarySample'] == True]
    return df

In [37]:
#normalize proportions for the regression

university_board_statistics_df['female_proportion'] = \
         university_board_statistics_df.apply(
             lambda row: row['female'] / row['total_members'] if row['total_members'] > 0 else 0,
             axis=1
         )

university_board_statistics_df['poc_proportion'] = \
         university_board_statistics_df.apply(
             lambda row: row['poc'] / row['total_ethnicity'] if row['total_ethnicity'] > 0 else 0,
             axis=1
         )

# Calculate the proportion of billionaires on the board
university_board_statistics_df['billionaire_proportion'] = (
    university_board_statistics_df['num_billionaires'] / 
    university_board_statistics_df['total_members'].replace(0, np.nan)
).fillna(0)

university_board_statistics_df = remove_non_samples(university_board_statistics_df)

university_board_statistics_df.to_csv(university_boards_statistics_path, index=False)


In [38]:
'''Logistic Regression'''

dependent_var = "female_president"
year_var = "Year"

# Note: We are going to convert "region" to a numeric variable later (as region_numeric).
independent_vars = [
    "admissions.admission_rate.overall", 
    "student.students_with_pell_grant", 
    "female_proportion",
    "billionaire_proportion",
    "total_members",
    "eigenvector",
    "degree",
    "student.demographics.women",
    "poc_proportion",
    "board_turnover",
    "control",
    "student.demographics.faculty.women",
    "region",
    # "strength"
]

# When dropping missing data, drop on the columns that exist in the original DataFrame.
# We exclude "region_numeric" (since it will be created from "region") and drop on "region" instead.
cols_to_check = [dependent_var] + independent_vars + [year_var, "region"]
regression_data = university_board_statistics_df.dropna(subset=cols_to_check).copy()

# Ensure the dependent variable is binary.
regression_data[dependent_var] = regression_data[dependent_var].astype(int)

# Convert the categorical 'region' into a numeric variable.
# Define a mapping for the four regions. (Adjust the mapping as needed.)
region_map = {"Northeast": 0, "Midwest": 1, "South": 2, "West": 3}
regression_data["region_numeric"] = regression_data["region"].map(region_map)

# Now, create dummy variables for Year only.
regression_data = pd.get_dummies(
    regression_data,
    columns=[year_var],
    drop_first=True
)

# Identify the dummy columns for Year.
year_dummies = [col for col in regression_data.columns if col.startswith(f"{year_var}_")]
regression_data[year_dummies] = regression_data[year_dummies].astype(int)

# Build the full predictor list.
# We remove the original "region" from independent_vars and add our new "region_numeric" instead.
predictor_vars = [var for var in independent_vars if var != "region"] + ["region_numeric"] + year_dummies

X = regression_data[predictor_vars]
y = regression_data[dependent_var]

# Convert "control" to a binary variable.
X["control"] = X["control"].map({"Public": 1, "Private": 0})

# Ensure all predictor columns are numeric.
X = X.apply(pd.to_numeric)

# Normalize continuous predictors.
continuous_vars = [
    "admissions.admission_rate.overall", 
    "student.students_with_pell_grant", 
    "female_proportion",
    "billionaire_proportion",  
    "total_members",
    "eigenvector",
    "degree",
    "student.demographics.women",
    "poc_proportion",
    "board_turnover",
    "control",
    "student.demographics.faculty.women"
    # "strength"
    # Note: "region_numeric" is treated as categorical, so we leave it as is.
]

scaler = StandardScaler()
vars_to_scale = [var for var in continuous_vars if var in X.columns]
X_scaled = X.copy()
X_scaled[vars_to_scale] = scaler.fit_transform(X_scaled[vars_to_scale])

# Ensure the scaled data is numeric.
X_scaled = X_scaled.apply(pd.to_numeric)

# Add an intercept.
X_scaled = sm.add_constant(X_scaled)

# Fit the logistic regression model.
logit_model = sm.Logit(y, X_scaled)
result = logit_model.fit()

# Create a DataFrame with odds ratios and p-values.
odds_ratios = pd.DataFrame({
    "Variable": X_scaled.columns,
    "Coefficient": result.params,
    "Odds Ratio": np.exp(result.params),
    "P-Value": result.pvalues
})

print(result.summary())
print("\nOdds Ratios and P-Values:")
print(tabulate(odds_ratios, headers="keys", tablefmt="grid"))

# Calculate Variance Inflation Factors (VIF).
if 'const' in X_scaled.columns:
    X_check = X_scaled.drop('const', axis=1)
else:
    X_check = X_scaled.copy()

vif_data = pd.DataFrame({
    "Variable": X_check.columns,
    "VIF": [variance_inflation_factor(X_check.values, i) for i in range(X_check.shape[1])]
})
print("\nVariance Inflation Factors (VIF):")
print(tabulate(vif_data, headers="keys", tablefmt="grid"))


C:\Users\tykun\AppData\Local\Temp\ipykernel_41712\1673689539.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["control"] = X["control"].map({"Public": 1, "Private": 0})


Optimization terminated successfully.
         Current function value: 0.339821
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:       female_president   No. Observations:                  837
Model:                          Logit   Df Residuals:                      819
Method:                           MLE   Df Model:                           17
Date:                Mon, 10 Feb 2025   Pseudo R-squ.:                  0.1414
Time:                        01:13:42   Log-Likelihood:                -284.43
converged:                       True   LL-Null:                       -331.28
Covariance Type:            nonrobust   LLR p-value:                 1.290e-12
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                 -1.6202      0.289    

In [39]:
# Define dependent and independent variables (for later reference)
dependent_var = "female_president"
year_var = "Year"

# (Keep your original full list for reference if needed)
# independent_vars = [
#     "admissions.admission_rate.overall", 
#     "student.students_with_pell_grant", 
#     "female_proportion",
#     "billionaire_proportion",
#     "total_members",
#     "eigenvector",
#     "degree",
#     "student.demographics.women",
#     "poc_proportion",
#     "board_turnover",
#     "control",
#     "student.demographics.faculty.women",
#     "region",
#     "strength"
# ]

# ---------------------------
# Preprocessing steps (unchanged)
# ---------------------------

# Drop rows with missing data
regression_data = university_board_statistics_df.dropna(
    subset=[dependent_var] + [
        "admissions.admission_rate.overall", 
        "student.students_with_pell_grant", 
        "female_proportion",
        "billionaire_proportion",
        "total_members",
        "eigenvector",
        "degree",
        "student.demographics.women",
        "poc_proportion",
        "board_turnover",
        "control",
        "student.demographics.faculty.women",
        "region",
        "strength"
    ] + [year_var]
).copy()

# Ensure binary dependent variable
regression_data[dependent_var] = regression_data[dependent_var].astype(int)

# Convert the categorical 'region' into a numeric variable.
region_map = {"Northeast": 0, "Midwest": 1, "South": 2, "West": 3}
regression_data["region"] = regression_data["region"].map(region_map)

# Create dummy variables for the year
regression_data = pd.get_dummies(
    regression_data,
    columns=[year_var],
    drop_first=True
)

# Ensure correct data type for year dummy variables
year_dummies = [col for col in regression_data.columns if col.startswith(f"{year_var}_")]
regression_data[year_dummies] = regression_data[year_dummies].astype(int)

# Convert "control" column to binary mapping
regression_data["control"] = regression_data["control"].map({"Public": 1, "Private": 0})

# Normalize continuous variables
continuous_vars = [
    "admissions.admission_rate.overall", 
    "student.students_with_pell_grant", 
    "female_proportion",
    "billionaire_proportion",  
    "total_members",
    "eigenvector",
    "degree",
    "student.demographics.women",
    "poc_proportion",
    "board_turnover",
    "control",
    "student.demographics.faculty.women",
    "strength"    # Added missing comma in the original code
]
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
vars_to_scale = [var for var in continuous_vars if var in regression_data.columns]
regression_data[vars_to_scale] = scaler.fit_transform(regression_data[vars_to_scale])

# ---------------------------
# New Model Specification (four models)
# ---------------------------

# Define predictor groups by specification.
# Model 1: Controls only (exclude region, non-network predictors, and network measures)
control_vars = [
    "admissions.admission_rate.overall", 
    "student.students_with_pell_grant", 
    "total_members",
    "board_turnover",
    "control",
    "student.demographics.faculty.women"
]

# Model 2: Add region.
model2_vars = control_vars + ["region"]

# Model 3: Add additional non-network predictors: billionaire_proportion, female_proportion, poc_proportion.
model3_vars = model2_vars + [
    "billionaire_proportion", 
    "female_proportion", 
    "poc_proportion"
]

# Model 4: Add network measures: eigenvector, degree, and strength.
model4_vars = model3_vars + ["eigenvector", "degree", "strength"]

# Create a list of model specifications.
model_specs = [control_vars, model2_vars, model3_vars, model4_vars]

# ---------------------------
# Fit multiple logistic regression models
# ---------------------------
import statsmodels.api as sm
from stargazer.stargazer import Stargazer

models = []
for predictors in model_specs:
    # Use the predictors plus the year dummy variables.
    X = regression_data[predictors + year_dummies]
    X = sm.add_constant(X)
    y = regression_data[dependent_var]
    model = sm.Logit(y, X).fit(disp=False)  # Set disp=False for quiet output
    models.append(model)

# Generate Stargazer output for multiple regression results
stargazer = Stargazer(models)
stargazer.title("Logistic Regression Results with Different Specifications")
stargazer.dependent_variable_name(dependent_var)

# Display the output as LaTeX
print(stargazer.render_latex())


\begin{table}[!htbp] \centering
  \caption{Logistic Regression Results with Different Specifications}
\begin{tabular}{@{\extracolsep{5pt}}lcccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{4}{c}{\textit{Dependent variable: female_president}} \
\cr \cline{2-5}
\\[-1.8ex] & (1) & (2) & (3) & (4) \\
\hline \\[-1.8ex]
 Year_2007 & -0.164$^{}$ & -0.172$^{}$ & -0.233$^{}$ & -0.224$^{}$ \\
& (0.321) & (0.323) & (0.337) & (0.337) \\
 Year_2009 & -0.169$^{}$ & -0.189$^{}$ & -0.316$^{}$ & -0.309$^{}$ \\
& (0.337) & (0.339) & (0.353) & (0.355) \\
 Year_2011 & -0.207$^{}$ & -0.233$^{}$ & -0.178$^{}$ & -0.195$^{}$ \\
& (0.352) & (0.354) & (0.366) & (0.376) \\
 Year_2013 & -0.349$^{}$ & -0.373$^{}$ & -0.378$^{}$ & -0.373$^{}$ \\
& (0.363) & (0.365) & (0.378) & (0.381) \\
 admissions.admission_rate.overall & -0.809$^{***}$ & -0.796$^{***}$ & -0.691$^{***}$ & -0.664$^{***}$ \\
& (0.137) & (0.137) & (0.145) & (0.150) \\
 billionaire_proportion & & & -0.266$^{*}$ & -0.271$^{*}$ \\
& & & (0.151) & 